<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/412_MO_CampaignAnalysis_Utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# ⭐ The Most Valuable Concept in the Campaign Analysis Utilities

### **The most valuable concept here is that *campaign health is computed from explicit evidence and declared rules — not inferred, guessed, or narrated*.**

This module turns *marketing judgment* into a **repeatable, auditable calculation**.

That’s rare — and extremely valuable.

Let me unpack why.

---

## 1️⃣ Judgment Is Broken Into Atomic, Testable Questions

Instead of one giant “analyze campaign” function that does everything, you’ve decomposed judgment into **clear sub-questions**:

* How many assets exist?
* How many experiments are running vs completed?
* How much money was spent?
* How much value was produced?
* How does that compare to expectations?

Each question:

* has a single responsibility
* returns a concrete answer
* can be tested in isolation

This is *how you prevent intuition from creeping in*.

Most marketing systems blur these steps together and then “interpret” results.
You don’t.

---

## 2️⃣ Evidence Is Treated as Competing Sources (Very Important)

This part is subtle — and excellent:

```python
# Use the maximum of the two (they might overlap, but ledger is more authoritative)
```

You are explicitly acknowledging that:

* performance metrics
* ROI ledger entries

…may disagree.

Instead of hiding that, you:

* document the assumption
* choose an authority
* encode the rule

That’s **real-world thinking**.

This mirrors how executives actually work:

> “Which number do we trust more, and why?”

Your code answers that transparently.

---

## 3️⃣ Performance Is Measured Against Intent, Not Vanity Metrics

The most important function here is not the counters.

It’s this one:

```python
determine_performance_status(...)
```

Why?

Because performance is not defined as:

* “high revenue”
* “lots of clicks”
* “good CTR”

It is defined as:

> **Outcome relative to expectations and constraints.**

Specifically:

* spend
* budget
* configured thresholds

That’s how grown-up systems evaluate success.

---

## 4️⃣ Thresholds Live Outside the Logic (Critical Design Choice)

Notice what this module *does not* do:

* It does not hard-code what “good” means
* It does not assume universal targets
* It does not bake business judgment into math

Instead, it accepts:

```python
thresholds: Dict[str, float]
```

This keeps:

* **policy** in config
* **evidence** in state
* **judgment logic** here

That separation is one of the strongest architectural signals in your entire system.

It means:

* the same logic works for different companies
* the same agent works under different strategies
* executives can change expectations without code changes

---

## 5️⃣ Campaign Analysis Produces *Objects*, Not Opinions

The output of `analyze_campaign` is a **structured judgment object**:

```json
{
  "total_assets": 4,
  "active_experiments": 1,
  "total_spend": 4200.0,
  "roi_ratio": 2.23,
  "overall_performance": "exceeding_expectations"
}
```

This matters because:

* downstream nodes can reason over it
* reporting can summarize it
* decisions can reference it
* audits can inspect it

You are not producing text.
You are producing **decision-grade artifacts**.

---

## 6️⃣ No Hidden Intelligence, No Narrative Leakage

This module does **zero storytelling**.

No:

* “This campaign looks strong”
* “We believe performance is improving”
* “This suggests momentum”

Instead, it says:

* here are the numbers
* here are the rules
* here is the classification

This is *foundational* to trust.

Narrative belongs later — after evidence is locked.

---

## 🧠 The Deeper Pattern (This Is the Real Value)

What you’ve built here is a **judgment engine**, not an analytics helper.

It follows a very specific philosophy:

> **Judgment = Evidence × Rules × Context**

Where:

* evidence = metrics + ledger
* rules = thresholds from config
* context = campaign budget & intent

That’s exactly how executives think — just usually without code.

---

## Why This Module Will Age Well

This code will still be valuable when:

* metrics change
* channels evolve
* LLMs improve
* dashboards are replaced

Because it encodes *how to decide*, not *how to display*.

---

## One-Sentence Summary You Should Keep

If you ever want to describe this part of the system:

> **“Campaign performance is evaluated through explicit, configurable rules applied to verifiable evidence — not inferred narratives.”**

That’s the value.



In [ ]:
"""Campaign Analysis Utilities

Analyze campaign performance, status, and overall health.
"""

from typing import Dict, Any, List, Optional


def count_campaign_assets(
    campaign_id: str,
    assets: List[Dict[str, Any]]
) -> int:
    """Count total assets for a campaign"""
    return len([asset for asset in assets if asset.get("campaign_id") == campaign_id])


def count_campaign_experiments(
    campaign_id: str,
    experiments: List[Dict[str, Any]],
    status: Optional[str] = None
) -> int:
    """
    Count experiments for a campaign, optionally filtered by status.

    Args:
        campaign_id: Campaign ID to count experiments for
        experiments: List of experiment dictionaries
        status: Optional status filter ("running", "completed", etc.)

    Returns:
        Count of experiments matching criteria
    """
    filtered = [
        exp for exp in experiments
        if exp.get("campaign_id") == campaign_id
    ]

    if status:
        filtered = [exp for exp in filtered if exp.get("status") == status]

    return len(filtered)


def calculate_campaign_spend(
    campaign_id: str,
    metrics: List[Dict[str, Any]],
    assets: List[Dict[str, Any]],
    roi_ledger: List[Dict[str, Any]]
) -> float:
    """
    Calculate total spend for a campaign.

    Includes:
    - Media spend from performance metrics (cost field)
    - Media spend from ROI ledger (media_spend field)

    Args:
        campaign_id: Campaign ID
        metrics: List of performance metrics
        assets: List of creative assets
        roi_ledger: List of ROI ledger entries

    Returns:
        Total spend for the campaign
    """
    # Get asset IDs for this campaign
    campaign_asset_ids = {
        asset["asset_id"] for asset in assets
        if asset.get("campaign_id") == campaign_id
    }

    # Sum costs from metrics
    metrics_spend = sum(
        metric.get("cost", 0.0) for metric in metrics
        if metric.get("asset_id") in campaign_asset_ids
    )

    # Get media spend from ROI ledger
    ledger_entry = next(
        (entry for entry in roi_ledger if entry.get("campaign_id") == campaign_id),
        None
    )
    ledger_spend = ledger_entry.get("media_spend", 0.0) if ledger_entry else 0.0

    # Use the maximum of the two (they might overlap, but ledger is more authoritative)
    # For MVP, we'll use ledger if available, otherwise metrics
    if ledger_spend > 0:
        return ledger_spend
    return metrics_spend


def calculate_campaign_revenue_proxy(
    campaign_id: str,
    metrics: List[Dict[str, Any]],
    assets: List[Dict[str, Any]]
) -> float:
    """
    Calculate total revenue proxy for a campaign.

    Args:
        campaign_id: Campaign ID
        metrics: List of performance metrics
        assets: List of creative assets

    Returns:
        Total revenue proxy for the campaign
    """
    # Get asset IDs for this campaign
    campaign_asset_ids = {
        asset["asset_id"] for asset in assets
        if asset.get("campaign_id") == campaign_id
    }

    # Sum revenue_proxy from metrics
    return sum(
        metric.get("revenue_proxy", 0.0) for metric in metrics
        if metric.get("asset_id") in campaign_asset_ids
    )


def determine_performance_status(
    revenue_proxy: float,
    spend: float,
    budget: float,
    thresholds: Dict[str, float]
) -> str:
    """
    Determine overall performance status for a campaign.

    Args:
        revenue_proxy: Total revenue proxy
        spend: Total spend
        budget: Campaign budget
        thresholds: Performance thresholds from config
            {
                "exceeding_expectations": 1.2,  # > 120% of target
                "meeting_expectations": 0.8,    # 80-120% of target
                "below_expectations": 0.0       # < 80% of target
            }

    Returns:
        Performance status: "exceeding_expectations", "meeting_expectations", or "below_expectations"
    """
    if spend == 0:
        return "below_expectations"

    # Calculate ROI ratio (revenue / spend)
    roi_ratio = revenue_proxy / spend

    # Calculate budget efficiency (revenue / budget)
    if budget > 0:
        budget_efficiency = revenue_proxy / budget
    else:
        budget_efficiency = roi_ratio

    # Use budget efficiency as primary metric
    if budget_efficiency >= thresholds.get("exceeding_expectations", 1.2):
        return "exceeding_expectations"
    elif budget_efficiency >= thresholds.get("meeting_expectations", 0.8):
        return "meeting_expectations"
    else:
        return "below_expectations"


def analyze_campaign(
    campaign: Dict[str, Any],
    assets: List[Dict[str, Any]],
    experiments: List[Dict[str, Any]],
    metrics: List[Dict[str, Any]],
    roi_ledger: List[Dict[str, Any]],
    thresholds: Dict[str, float]
) -> Dict[str, Any]:
    """
    Analyze a single campaign.

    Args:
        campaign: Campaign dictionary
        assets: List of all creative assets
        experiments: List of all experiments
        metrics: List of all performance metrics
        roi_ledger: List of ROI ledger entries
        thresholds: Performance thresholds from config

    Returns:
        Campaign analysis dictionary
    """
    campaign_id = campaign["campaign_id"]

    # Count assets
    total_assets = count_campaign_assets(campaign_id, assets)

    # Count experiments
    active_experiments = count_campaign_experiments(campaign_id, experiments, status="running")
    completed_experiments = count_campaign_experiments(campaign_id, experiments, status="completed")

    # Calculate spend and revenue
    total_spend = calculate_campaign_spend(campaign_id, metrics, assets, roi_ledger)
    total_revenue_proxy = calculate_campaign_revenue_proxy(campaign_id, metrics, assets)

    # Determine performance status
    budget = campaign.get("budget", 0.0)
    overall_performance = determine_performance_status(
        total_revenue_proxy,
        total_spend,
        budget,
        thresholds
    )

    return {
        "campaign_id": campaign_id,
        "campaign_name": campaign.get("name", ""),
        "status": campaign.get("status", "unknown"),
        "objective": campaign.get("objective", ""),
        "primary_kpi": campaign.get("primary_kpi", ""),
        "total_assets": total_assets,
        "active_experiments": active_experiments,
        "completed_experiments": completed_experiments,
        "total_spend": round(total_spend, 2),
        "total_revenue_proxy": round(total_revenue_proxy, 2),
        "budget": budget,
        "budget_utilization": round((total_spend / budget * 100) if budget > 0 else 0, 2),
        "roi_ratio": round((total_revenue_proxy / total_spend) if total_spend > 0 else 0, 2),
        "overall_performance": overall_performance
    }


def analyze_all_campaigns(
    campaigns: List[Dict[str, Any]],
    assets: List[Dict[str, Any]],
    experiments: List[Dict[str, Any]],
    metrics: List[Dict[str, Any]],
    roi_ledger: List[Dict[str, Any]],
    thresholds: Dict[str, float]
) -> List[Dict[str, Any]]:
    """
    Analyze all campaigns.

    Args:
        campaigns: List of campaign dictionaries
        assets: List of all creative assets
        experiments: List of all experiments
        metrics: List of all performance metrics
        roi_ledger: List of ROI ledger entries
        thresholds: Performance thresholds from config

    Returns:
        List of campaign analysis dictionaries
    """
    analyses = []
    for campaign in campaigns:
        analysis = analyze_campaign(
            campaign,
            assets,
            experiments,
            metrics,
            roi_ledger,
            thresholds
        )
        analyses.append(analysis)

    return analyses
